In [2]:
def get_data(dtype="train"):
    fp = open("facedata/facedata"+dtype, "r")
    count = 0
    data_point = [[0 for _ in range(70)] for _ in range(70)]
    data = []
    for a in fp:
        count += 1
        if count == 1:
            data_point = [[0 for _ in range(70)] for _ in range(70)]
            line = a.replace("\n","")
            if "+" in line or "#" in line:
                c  = 0
                for i in str(line):
                    if i == '+' or i == '#':
                        data_point[count-1][c] = 1
                    c += 1
        elif count == 70: 
            line = a.replace("\n","")
            if "+" in line or "#" in line:
                c  = 0 
                for i in str(line):
                    if i == '+' or i == '#':
                        data_point[count-1][c] = 1
                    c += 1
            count = 0
            data.append(data_point)
        else:
            line = a.replace("\n","")
            if "+" in line or "#" in line:
                c  = 0 
                for i in str(line):
                    if i == '+' or i == '#':
                        data_point[count-1][c] = 1
                    c += 1        
    fp.close()
    actual_y = []
    fp = open("facedata/facedata"+dtype+"labels","r")
    for a in fp:
        actual_y.append(int(a.replace("\n","")))
    fp.close()
    return data, actual_y

In [4]:
import pandas as pd
columns = ["Data%"]

for i in range(1,51):
    columns.append('Prediction_Error_Test_Trail_'+str(i))

for i in range(1,51):
    columns.append('Prediction_Error_Validation_Trail_'+str(i))

for i in range(1,51):
    columns.append('Training_Time_Trail_'+str(i))
df = pd.DataFrame(columns = columns)

df['Data%'] = [x for x in range(10,101, 10)]
df = df.set_index('Data%')
#df.loc[20]['Correct_Predictions_Test_Trail_1'] = 333
df

,Prediction_Error_Test_Trail_1,Prediction_Error_Test_Trail_2,Prediction_Error_Test_Trail_3,Prediction_Error_Test_Trail_4,Prediction_Error_Test_Trail_5,Prediction_Error_Test_Trail_6,Prediction_Error_Test_Trail_7,Prediction_Error_Test_Trail_8,Prediction_Error_Test_Trail_9,Prediction_Error_Test_Trail_10,...,Training_Time_Trail_41,Training_Time_Trail_42,Training_Time_Trail_43,Training_Time_Trail_44,Training_Time_Trail_45,Training_Time_Trail_46,Training_Time_Trail_47,Training_Time_Trail_48,Training_Time_Trail_49,Training_Time_Trail_50
Data%,,,,,,,,,,,,,,,,,,,,,
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
from util_faces import *
import sys
import time
import random

training_data, training_y = get_data() 
for p in range(10, 101, 10):
    
    training_percent = p
    
    for trail in range(1,51):
        print('Percentage',p,'Trail',trail, 'Started')
        #print("\n////////////////////////////////")
        #print("// ⦿ training data used:", str(training_percent)+ "%")


        for i in range(len(training_data)):
            training_data[i].append(training_y[i])

        random.shuffle(training_data)

        training_y = []
        for i in range(len(training_data)):
            training_y.append(training_data[i].pop())
            
        
        st = time.time()
        features = []

        inos = [x for x in range(451*training_percent//100)]
        prob_table_face = [[[0 for _ in range(101)] for _ in range(7)] for _ in range(7)]
        prob_table_not_face = [[[0 for _ in range(101)] for _ in range(7)] for _ in range(7)]
        num_faces = 0

        for ino in inos:
            feature = [[0 for _ in range(7)] for _ in range(7)]
            for x in range(7):
                for y in range(7):
                    i = 10*x
                    j = 10*y
                    for a in range(i, i+10):
                        for b in range(j, j+10):
                            feature[x][y] += training_data[ino][a][b]
            features.append(feature)

        for ino in inos:    
            if training_y[ino] == 1:
                num_faces += 1
                for x in range(7):
                    for y in range(7):
                        prob_table_face[x][y][features[ino][x][y]] += 1
            else:
                for x in range(7):
                    for y in range(7):
                            prob_table_not_face[x][y][features[ino][x][y]] += 1
                            
        et = time.time()
        train_time = et-st
        df.loc[p]['Training_Time_Trail_'+str(trail)] = train_time

        test_accuracy = 0
        test_data, test_y = get_data(dtype="test")
        features_test = []
        for ino, label in enumerate(test_y):
            feature = [[0 for _ in range(7)] for _ in range(7)]
            for x in range(7):
                for y in range(7):
                    i = 10*x
                    j = 10*y
                    for a in range(i, i+10):
                        for b in range(j, j+10):
                            feature[x][y] += test_data[ino][a][b]
            features_test.append(feature)

        for ino in range(len(test_data)):
            pface = num_faces/len(inos)
            pnotface = 1 - pface
            for x in range(7):
                for y in range(7):
                    factor = prob_table_face[x][y][features_test[ino][x][y]]/sum(prob_table_face[x][y]) if prob_table_face[x][y][features_test[ino][x][y]] != 0 else 1/num_faces
                    pface *= factor
                    factor2 = prob_table_not_face[x][y][features_test[ino][x][y]]/sum(prob_table_not_face[x][y]) if prob_table_not_face[x][y][features_test[ino][x][y]] != 0 else 1/(len(inos)-num_faces)
                    pnotface *= factor2

            if pface > pnotface and test_y[ino] == 1:
                test_accuracy += 1
            elif pface < pnotface and test_y[ino] == 0:
                test_accuracy += 1

        test_accuracy /= len(test_data)
        pred_error = 1-test_accuracy
        df.loc[p]['Prediction_Error_Test_Trail_'+str(trail)] = pred_error
        #print("// ➔ Test Accuracy:", str(round(test_accuracy*100))+"%")

        validation_accuracy = 0
        validation_data, validation_y = get_data(dtype="validation")
        features_validation = []
        for ino, label in enumerate(validation_y):
            feature = [[0 for _ in range(7)] for _ in range(7)]
            for x in range(7):
                for y in range(7):
                    i = 10*x
                    j = 10*y
                    for a in range(i, i+10):
                        for b in range(j, j+10):
                            feature[x][y] += validation_data[ino][a][b]
            features_validation.append(feature)

        for ino in range(len(validation_data)):
            pface = num_faces/len(inos)
            pnotface = 1 - pface
            for x in range(7):
                for y in range(7):
                    factor = prob_table_face[x][y][features_validation[ino][x][y]]/sum(prob_table_face[x][y]) if prob_table_face[x][y][features_validation[ino][x][y]] != 0 else 1/num_faces
                    pface *= factor
                    factor2 = prob_table_not_face[x][y][features_validation[ino][x][y]]/sum(prob_table_not_face[x][y]) if prob_table_not_face[x][y][features_validation[ino][x][y]] != 0 else 1/(len(inos)-num_faces)
                    pnotface *= factor2

            if pface > pnotface and validation_y[ino] == 1:
                validation_accuracy += 1
            elif pface < pnotface and validation_y[ino] == 0:
                validation_accuracy += 1

        validation_accuracy /= len(validation_data)
        pred_error = 1-validation_accuracy
        #print("// ➔ validation Accuracy:", str(round(validation_accuracy*100))+"%")
        df.loc[p]['Prediction_Error_Validation_Trail_'+str(trail)] = pred_error
        #print("// ➔ Validation Accuracy:", str(round(validation_accuracy*100))+"%")
        print('Percentage',p,'Trail',trail, 'Ended')
        print()
        #print("////////////////////////////////")

Percentage 10 Trail 1 Started
Percentage 10 Trail 1 Ended

Percentage 10 Trail 2 Started
Percentage 10 Trail 2 Ended

Percentage 10 Trail 3 Started
Percentage 10 Trail 3 Ended

Percentage 10 Trail 4 Started
Percentage 10 Trail 4 Ended

Percentage 10 Trail 5 Started
Percentage 10 Trail 5 Ended

Percentage 10 Trail 6 Started
Percentage 10 Trail 6 Ended

Percentage 10 Trail 7 Started
Percentage 10 Trail 7 Ended

Percentage 10 Trail 8 Started
Percentage 10 Trail 8 Ended

Percentage 10 Trail 9 Started
Percentage 10 Trail 9 Ended

Percentage 10 Trail 10 Started
Percentage 10 Trail 10 Ended

Percentage 10 Trail 11 Started
Percentage 10 Trail 11 Ended

Percentage 10 Trail 12 Started
Percentage 10 Trail 12 Ended

Percentage 10 Trail 13 Started
Percentage 10 Trail 13 Ended

Percentage 10 Trail 14 Started
Percentage 10 Trail 14 Ended

Percentage 10 Trail 15 Started
Percentage 10 Trail 15 Ended

Percentage 10 Trail 16 Started
Percentage 10 Trail 16 Ended

Percentage 10 Trail 17 Started
Percentage 

Percentage 30 Trail 36 Ended

Percentage 30 Trail 37 Started
Percentage 30 Trail 37 Ended

Percentage 30 Trail 38 Started
Percentage 30 Trail 38 Ended

Percentage 30 Trail 39 Started
Percentage 30 Trail 39 Ended

Percentage 30 Trail 40 Started
Percentage 30 Trail 40 Ended

Percentage 30 Trail 41 Started
Percentage 30 Trail 41 Ended

Percentage 30 Trail 42 Started
Percentage 30 Trail 42 Ended

Percentage 30 Trail 43 Started
Percentage 30 Trail 43 Ended

Percentage 30 Trail 44 Started
Percentage 30 Trail 44 Ended

Percentage 30 Trail 45 Started
Percentage 30 Trail 45 Ended

Percentage 30 Trail 46 Started
Percentage 30 Trail 46 Ended

Percentage 30 Trail 47 Started
Percentage 30 Trail 47 Ended

Percentage 30 Trail 48 Started
Percentage 30 Trail 48 Ended

Percentage 30 Trail 49 Started
Percentage 30 Trail 49 Ended

Percentage 30 Trail 50 Started
Percentage 30 Trail 50 Ended

Percentage 40 Trail 1 Started
Percentage 40 Trail 1 Ended

Percentage 40 Trail 2 Started
Percentage 40 Trail 2 Ended

Percentage 60 Trail 22 Ended

Percentage 60 Trail 23 Started
Percentage 60 Trail 23 Ended

Percentage 60 Trail 24 Started
Percentage 60 Trail 24 Ended

Percentage 60 Trail 25 Started
Percentage 60 Trail 25 Ended

Percentage 60 Trail 26 Started
Percentage 60 Trail 26 Ended

Percentage 60 Trail 27 Started
Percentage 60 Trail 27 Ended

Percentage 60 Trail 28 Started
Percentage 60 Trail 28 Ended

Percentage 60 Trail 29 Started
Percentage 60 Trail 29 Ended

Percentage 60 Trail 30 Started
Percentage 60 Trail 30 Ended

Percentage 60 Trail 31 Started
Percentage 60 Trail 31 Ended

Percentage 60 Trail 32 Started
Percentage 60 Trail 32 Ended

Percentage 60 Trail 33 Started
Percentage 60 Trail 33 Ended

Percentage 60 Trail 34 Started
Percentage 60 Trail 34 Ended

Percentage 60 Trail 35 Started
Percentage 60 Trail 35 Ended

Percentage 60 Trail 36 Started
Percentage 60 Trail 36 Ended

Percentage 60 Trail 37 Started
Percentage 60 Trail 37 Ended

Percentage 60 Trail 38 Started
Percentage 60 Trail 38 E

Percentage 90 Trail 8 Ended

Percentage 90 Trail 9 Started
Percentage 90 Trail 9 Ended

Percentage 90 Trail 10 Started
Percentage 90 Trail 10 Ended

Percentage 90 Trail 11 Started
Percentage 90 Trail 11 Ended

Percentage 90 Trail 12 Started
Percentage 90 Trail 12 Ended

Percentage 90 Trail 13 Started
Percentage 90 Trail 13 Ended

Percentage 90 Trail 14 Started
Percentage 90 Trail 14 Ended

Percentage 90 Trail 15 Started
Percentage 90 Trail 15 Ended

Percentage 90 Trail 16 Started
Percentage 90 Trail 16 Ended

Percentage 90 Trail 17 Started
Percentage 90 Trail 17 Ended

Percentage 90 Trail 18 Started
Percentage 90 Trail 18 Ended

Percentage 90 Trail 19 Started
Percentage 90 Trail 19 Ended

Percentage 90 Trail 20 Started
Percentage 90 Trail 20 Ended

Percentage 90 Trail 21 Started
Percentage 90 Trail 21 Ended

Percentage 90 Trail 22 Started
Percentage 90 Trail 22 Ended

Percentage 90 Trail 23 Started
Percentage 90 Trail 23 Ended

Percentage 90 Trail 24 Started
Percentage 90 Trail 24 Ende

In [7]:
df

,Prediction_Error_Test_Trail_1,Prediction_Error_Test_Trail_2,Prediction_Error_Test_Trail_3,Prediction_Error_Test_Trail_4,Prediction_Error_Test_Trail_5,Prediction_Error_Test_Trail_6,Prediction_Error_Test_Trail_7,Prediction_Error_Test_Trail_8,Prediction_Error_Test_Trail_9,Prediction_Error_Test_Trail_10,...,Training_Time_Trail_41,Training_Time_Trail_42,Training_Time_Trail_43,Training_Time_Trail_44,Training_Time_Trail_45,Training_Time_Trail_46,Training_Time_Trail_47,Training_Time_Trail_48,Training_Time_Trail_49,Training_Time_Trail_50
Data%,,,,,,,,,,,,,,,,,,,,,
10,0.493333,0.46,0.513333,0.4,0.313333,0.393333,0.313333,0.293333,0.506667,0.426667,...,0.0418861,0.0418856,0.0398932,0.041887,0.0418875,0.0418866,0.0418906,0.044878,0.0418878,0.0418551
20,0.233333,0.293333,0.426667,0.233333,0.28,0.313333,0.333333,0.28,0.166667,0.38,...,0.0827782,0.0817795,0.0807512,0.0827768,0.081825,0.0797861,0.0808094,0.0817809,0.0807459,0.0828145
30,0.266667,0.273333,0.3,0.253333,0.226667,0.213333,0.273333,0.24,0.34,0.213333,...,0.120677,0.123668,0.130681,0.128658,0.121673,0.123664,0.122639,0.122672,0.123668,0.127658
40,0.186667,0.213333,0.253333,0.22,0.16,0.18,0.233333,0.153333,0.24,0.226667,...,0.163562,0.16057,0.167583,0.16252,0.173535,0.166557,0.202458,0.16459,0.168548,0.167518
50,0.2,0.206667,0.2,0.173333,0.193333,0.16,0.226667,0.18,0.133333,0.186667,...,0.20644,0.201454,0.202427,0.202458,0.216415,0.214425,0.203488,0.210439,0.204415,0.205451
60,0.16,0.14,0.193333,0.153333,0.153333,0.14,0.126667,0.166667,0.14,0.133333,...,0.249333,0.253286,0.253354,0.255317,0.244377,0.244311,0.26177,0.258309,0.262141,0.243349
70,0.12,0.14,0.133333,0.113333,0.166667,0.113333,0.12,0.153333,0.14,0.08,...,0.290221,0.307207,0.309141,0.289257,0.299199,0.284239,0.291221,0.290257,0.293215,0.294213
80,0.133333,0.113333,0.1,0.12,0.12,0.0933333,0.126667,0.113333,0.133333,0.113333,...,0.337342,0.334107,0.337099,0.334136,0.331082,0.33211,0.33215,0.328119,0.325129,0.330149
90,0.133333,0.133333,0.12,0.126667,0.113333,0.113333,0.126667,0.12,0.12,0.106667,...,0.379982,0.373999,0.376991,0.368026,0.451759,0.387963,0.371008,0.372004,0.371043,0.381978


In [17]:
df.to_csv('Faces_Bayes.csv')

In [14]:
df['Average_Training_Time'] = df.iloc[:,101:151].mean(axis=1)
df['Average_Prediction_Error_Test'] = df.iloc[:,:51].mean(axis=1)
df['Average_Prediction_Error_Validation'] = df.iloc[:,51:101].mean(axis=1)
df

,Prediction_Error_Test_Trail_1,Prediction_Error_Test_Trail_2,Prediction_Error_Test_Trail_3,Prediction_Error_Test_Trail_4,Prediction_Error_Test_Trail_5,Prediction_Error_Test_Trail_6,Prediction_Error_Test_Trail_7,Prediction_Error_Test_Trail_8,Prediction_Error_Test_Trail_9,Prediction_Error_Test_Trail_10,...,Training_Time_Trail_44,Training_Time_Trail_45,Training_Time_Trail_46,Training_Time_Trail_47,Training_Time_Trail_48,Training_Time_Trail_49,Training_Time_Trail_50,Average_Training_Time,Average_Prediction_Error_Test,Average_Prediction_Error_Validation
Data%,,,,,,,,,,,,,,,,,,,,,
10,0.493333,0.46,0.513333,0.4,0.313333,0.393333,0.313333,0.293333,0.506667,0.426667,...,0.041887,0.0418875,0.0418866,0.0418906,0.044878,0.0418878,0.0418551,0.041966,0.393104,0.374506
20,0.233333,0.293333,0.426667,0.233333,0.28,0.313333,0.333333,0.28,0.166667,0.38,...,0.0827768,0.081825,0.0797861,0.0808094,0.0817809,0.0807459,0.0828145,0.082335,0.310901,0.296718
30,0.266667,0.273333,0.3,0.253333,0.226667,0.213333,0.273333,0.24,0.34,0.213333,...,0.128658,0.121673,0.123664,0.122639,0.122672,0.123668,0.127658,0.124333,0.236062,0.230673
40,0.186667,0.213333,0.253333,0.22,0.16,0.18,0.233333,0.153333,0.24,0.226667,...,0.16252,0.173535,0.166557,0.202458,0.16459,0.168548,0.167518,0.167276,0.200249,0.202606
50,0.2,0.206667,0.2,0.173333,0.193333,0.16,0.226667,0.18,0.133333,0.186667,...,0.202458,0.216415,0.214425,0.203488,0.210439,0.204415,0.205451,0.209399,0.169139,0.177864
60,0.16,0.14,0.193333,0.153333,0.153333,0.14,0.126667,0.166667,0.14,0.133333,...,0.255317,0.244377,0.244311,0.26177,0.258309,0.262141,0.243349,0.249807,0.146524,0.161957
70,0.12,0.14,0.133333,0.113333,0.166667,0.113333,0.12,0.153333,0.14,0.08,...,0.289257,0.299199,0.284239,0.291221,0.290257,0.293215,0.294213,0.291580,0.131036,0.148662
80,0.133333,0.113333,0.1,0.12,0.12,0.0933333,0.126667,0.113333,0.133333,0.113333,...,0.334136,0.331082,0.33211,0.33215,0.328119,0.325129,0.330149,0.333300,0.120383,0.138968
90,0.133333,0.133333,0.12,0.126667,0.113333,0.113333,0.126667,0.12,0.12,0.106667,...,0.368026,0.451759,0.387963,0.371008,0.372004,0.371043,0.381978,0.378389,0.119338,0.133793


In [15]:
df['Standard_Deviation_Test_Error'] = df.iloc[:, :51].std(axis=1)
df['Standard_Deviation_Validation_Error'] = df.iloc[:,51:101].std(axis=1)
df

,Prediction_Error_Test_Trail_1,Prediction_Error_Test_Trail_2,Prediction_Error_Test_Trail_3,Prediction_Error_Test_Trail_4,Prediction_Error_Test_Trail_5,Prediction_Error_Test_Trail_6,Prediction_Error_Test_Trail_7,Prediction_Error_Test_Trail_8,Prediction_Error_Test_Trail_9,Prediction_Error_Test_Trail_10,...,Training_Time_Trail_46,Training_Time_Trail_47,Training_Time_Trail_48,Training_Time_Trail_49,Training_Time_Trail_50,Average_Training_Time,Average_Prediction_Error_Test,Average_Prediction_Error_Validation,Standard_Deviation_Test_Error,Standard_Deviation_Validation_Error
Data%,,,,,,,,,,,,,,,,,,,,,
10,0.493333,0.46,0.513333,0.4,0.313333,0.393333,0.313333,0.293333,0.506667,0.426667,...,0.0418866,0.0418906,0.044878,0.0418878,0.0418551,0.041966,0.393104,0.374506,0.088393,0.102866
20,0.233333,0.293333,0.426667,0.233333,0.28,0.313333,0.333333,0.28,0.166667,0.38,...,0.0797861,0.0808094,0.0817809,0.0807459,0.0828145,0.082335,0.310901,0.296718,0.073167,0.082921
30,0.266667,0.273333,0.3,0.253333,0.226667,0.213333,0.273333,0.24,0.34,0.213333,...,0.123664,0.122639,0.122672,0.123668,0.127658,0.124333,0.236062,0.230673,0.041867,0.038195
40,0.186667,0.213333,0.253333,0.22,0.16,0.18,0.233333,0.153333,0.24,0.226667,...,0.166557,0.202458,0.16459,0.168548,0.167518,0.167276,0.200249,0.202606,0.031385,0.022269
50,0.2,0.206667,0.2,0.173333,0.193333,0.16,0.226667,0.18,0.133333,0.186667,...,0.214425,0.203488,0.210439,0.204415,0.205451,0.209399,0.169139,0.177864,0.025435,0.017125
60,0.16,0.14,0.193333,0.153333,0.153333,0.14,0.126667,0.166667,0.14,0.133333,...,0.244311,0.26177,0.258309,0.262141,0.243349,0.249807,0.146524,0.161957,0.019416,0.017570
70,0.12,0.14,0.133333,0.113333,0.166667,0.113333,0.12,0.153333,0.14,0.08,...,0.284239,0.291221,0.290257,0.293215,0.294213,0.291580,0.131036,0.148662,0.018004,0.024546
80,0.133333,0.113333,0.1,0.12,0.12,0.0933333,0.126667,0.113333,0.133333,0.113333,...,0.33211,0.33215,0.328119,0.325129,0.330149,0.333300,0.120383,0.138968,0.013875,0.029356
90,0.133333,0.133333,0.12,0.126667,0.113333,0.113333,0.126667,0.12,0.12,0.106667,...,0.387963,0.371008,0.372004,0.371043,0.381978,0.378389,0.119338,0.133793,0.009446,0.035711
